In [ ]:
import json
import pprint
import pandas as pd
import plotly.express as px

In [ ]:
try:
    with open('bundeslaender.geojson', 'r', encoding='utf-8') as f:
        geojson_bundeslaender = json.load(f)
    print("GeoJSON-Datei erfolgreich geladen.")
except FileNotFoundError:
    print("Fehler: 'bundeslaender.geojson' nicht gefunden. Bitte den korrekten Pfad angeben.")

In [ ]:
#'properties': {'id': 500496,
  #  'schluessel': '06',
    #'gen': 'Hessen',
    #'jahr': 2023,

#geojson_bundeslaender


In [ ]:
geojson_bundeslaender.keys()

In [ ]:
#featureidkey = 'schluessel' in plotly
#location im df = 'schluessel'

schluessel=[]
bundesland=[]
for land in geojson_bundeslaender['features']:
    schluessel.append(land['properties']['schluessel'])
    bundesland.append(land['properties']["gen"])

In [ ]:
schluessel

In [ ]:
len(geojson_bundeslaender['features'])

In [ ]:
bundesland


In [ ]:
df=pd.DataFrame({"Bundesland":bundesland,"schluessel":schluessel})


In [ ]:
df

In [ ]:
df.info()

In [ ]:
import json
import fiona
from shapely.geometry import shape, mapping, Polygon, MultiPolygon
from fiona.crs import from_epsg
from pyproj import Transformer

In [ ]:
# Umwandlung GEOjson in passendes Format

In [ ]:
input_geojson_path = 'bundeslaender.geojson'  # <-- HIER DEN KORREKTEN NAMEN IHRER DATEI EINSETZEN
output_geojson_path = 'bundeslaender_wgs84.geojson' # Die neue Datei im WGS84-Format

try:
    with fiona.open(input_geojson_path, 'r') as source:
        source_crs = source.crs
        print(f"Original CRS: {source_crs}")

        target_crs = from_epsg(4326) # EPSG:4326 for WGS84 (lat/lon)
        print(f"Ziel CRS: {target_crs}")

        # Pyproj Transformer für die Umwandlung
        # always_xy=True stellt sicher, dass die Reihenfolge der Koordinaten (lon, lat) ist,
        # was für GeoJSON und den späteren Plotly-Gebrauch wichtig ist.
        transformer = Transformer.from_crs(source_crs, target_crs, always_xy=True)

        new_schema = source.schema
        # Der Geometrie-Typ sollte im Schema korrekt übernommen werden,
        # aber stellen Sie sicher, dass er mit dem übereinstimmt, was Sie transformieren.
        # Beispiel: new_schema['geometry'] = 'MultiPolygon' wenn die gesamte Datei nur Multipolygone hat.
        # Wenn es eine Mischung ist, ist source.schema['geometry'] oft 'Unknown' oder 'GeometryCollection',
        # was ok ist, da wir die spezifischen Typen behandeln.

        with fiona.open(
            output_geojson_path,
            'w',
            driver='GeoJSON',
            crs=target_crs,
            schema=new_schema
        ) as sink:
            for feature in source:
                geom = shape(feature['geometry'])
                transformed_geom = None

                # Funktion zum Transformieren der Koordinaten eines Rings
                def transform_coords(coords):
                    return [transformer.transform(x, y) for x, y in coords]

                if geom.geom_type == 'Polygon':
                    # Transformiere Exterior und alle Interior rings
                    exterior_coords = transform_coords(geom.exterior.coords)
                    interior_coords_list = [transform_coords(interior.coords) for interior in geom.interiors]
                    transformed_geom = Polygon(exterior_coords, interior_coords_list)

                elif geom.geom_type == 'MultiPolygon':
                    transformed_polygons = []
                    for poly in geom.geoms: # Iteriere durch die einzelnen Polygone im MultiPolygon
                        exterior_coords = transform_coords(poly.exterior.coords)
                        interior_coords_list = [transform_coords(interior.coords) for interior in poly.interiors]
                        transformed_polygons.append(Polygon(exterior_coords, interior_coords_list))
                    
                    transformed_geom = MultiPolygon(transformed_polygons)

                elif geom.geom_type in ['Point', 'LineString', 'MultiPoint', 'MultiLineString', 'GeometryCollection']:
                    # Optional: Handhabung für andere Geometrie-Typen, falls sie in Ihrer Datei vorkommen
                    # Hier ist es am besten, die Transformation über mapping(geom.transform(transformer.transform)) zu versuchen
                    # oder eine spezifische Logik zu implementieren. Für Bundesländer sind aber Polygon/MultiPolygon am relevantesten.
                    print(f"Info: Geometrie-Typ '{geom.geom_type}' für Feature ID {feature.get('id', 'N/A')} gefunden. Falls dies nicht ein Polygon/MultiPolygon ist, könnte es eine speziellere Transformation benötigen.")
                    # Für dieses Beispiel überspringen wir Nicht-Polygon-Typen, um den Fokus zu behalten.
                    continue
                else:
                    print(f"Warnung: Unbekannter Geometrie-Typ '{geom.geom_type}' für Feature ID {feature.get('id', 'N/A')}. Überspringe Feature.")
                    continue # Überspringe dieses Feature, wenn der Typ nicht Polygon/MultiPolygon ist

                # Feature mit der neuen Geometrie schreiben
                if transformed_geom:
                    sink.write({
                        'geometry': mapping(transformed_geom),
                        'properties': feature['properties']
                    })
    print(f"GeoJSON erfolgreich nach WGS84 umgewandelt und gespeichert als: {output_geojson_path}")

except FileNotFoundError:
    print(f"Fehler: Die Datei '{input_geojson_path}' wurde nicht gefunden.")
    print("Bitte stellen Sie sicher, dass der Pfad korrekt ist.")
except Exception as e:
    print(f"Ein unerwarteter Fehler ist aufgetreten während der Reprojektion: {e}")
    # Zusätzliche Debug-Info
    import traceback
    traceback.print_exc()


In [ ]:
df_2=pd.read_csv("../../data/electric_vehicles/bestand_bland.csv",delimiter=";",dtype={"schluessel":"str"})

In [ ]:
df_2.info()

In [ ]:
df_2

In [ ]:
df_3=pd.merge(df,df_2,on="schluessel")

In [ ]:
df_3
print(df_3.columns)

In [ ]:
df_3.rename(columns={"sun_cars":"sum_cars"},inplace=True)

In [ ]:
df_3.head()

In [ ]:
df_3.to_csv("../../data/electric_vehicles/bestand_bland_processed.csv")

In [ ]:
with open('bundeslaender_wgs84.geojson', 'r', encoding='utf-8') as f:
    geojson_bundeslaender_wgs84 = json.load(f)
    print("Konvertierte GeoJSON-Datei 'bundeslaender_wgs84.geojson' erfolgreich geladen.")

In [ ]:
fig = px.choropleth_map(df_3,
                           geojson=geojson_bundeslaender_wgs84, # Die konvertierte GeoJSON-Datei
                           locations="schluessel", # Spalte im DataFrame, die den 2-stelligen Schlüssel enthält
                           featureidkey="properties.schluessel", # Pfad zum 'schluessel' im GeoJSON-Feature
                                                                 # (wie in Ihrer GeoJSON-Struktur gesehen: 'properties': {'schluessel': '06', ...})
                           color="sum_cars",               # Spalte, die die Farbe der Region bestimmt
                           color_continuous_scale="Blues",  # Farbskala (z.B. "Viridis", "Plasma", "Jet", "Greens", "Blues")
                           range_color=(df_3["sum_cars"].min(), df_3["sum_cars"].max()), # Stellen Sie sicher, dass die Farbskala die volle Bandbreite abdeckt
                          # mapbox_style="carto-positron",     # Basiskarte (z.B. "open-street-map", "carto-positron", "stamen-terrain")
                           zoom=4.5,                          # Zoom-Level für Deutschland (ca. 4.5 - 5.5)
                           center={"lat": 51.0, "lon": 10.0}, # Zentraler Punkt für Deutschland
                           opacity=0.8,                       # Deckkraft der eingefärbten Regionen
                           hover_name="Bundesland",      # Was im Tooltip als Haupttitel angezeigt wird
                           hover_data={'sum_cars': ':.0f'}, # Optional: Formatiert die Anzeige der PKW_Bestand im Tooltip (als ganze Zahl)
                           title="PKW-Bestand pro Bundesland in Deutschland" # Titel der Karte
                          )


fig.show()


In [ ]:
df=pd.read_csv("../../data/electric_vehicles/bestand_bland_processed.csv")

In [ ]:
df["schluessel"]=df["schluessel"].map({1:"01",2:"02",3:"03",4:"04",5:"05",6:"06",7:"07",8:"08",9:"09",10:"10",11:"11",12:"12",13:"13",14:"14",15:"15",16:"16"})

In [ ]:
df

In [ ]:
df["% BEV"]=(df["e_cars"]/df["sum_cars"])*100

In [ ]:
fig = px.choropleth_map(df,
                           geojson=geojson_bundeslaender_wgs84, # Die konvertierte GeoJSON-Datei
                           locations="schluessel", # Spalte im DataFrame, die den 2-stelligen Schlüssel enthält
                           featureidkey="properties.schluessel", # Pfad zum 'schluessel' im GeoJSON-Feature
                                                                 # (wie in Ihrer GeoJSON-Struktur gesehen: 'properties': {'schluessel': '06', ...})
                           color="% BEV",               # Spalte, die die Farbe der Region bestimmt
                           color_continuous_scale="Blues",  # Farbskala (z.B. "Viridis", "Plasma", "Jet", "Greens", "Blues")
                           range_color=(df["% BEV"].min(), df["% BEV"].max()), # Stellen Sie sicher, dass die Farbskala die volle Bandbreite abdeckt
                          # mapbox_style="carto-positron",     # Basiskarte (z.B. "open-street-map", "carto-positron", "stamen-terrain")
                           zoom=5,                          # Zoom-Level für Deutschland (ca. 4.5 - 5.5)
                           center={"lat": 51.0, "lon": 10.0}, # Zentraler Punkt für Deutschland
                           opacity=0.8,                       # Deckkraft der eingefärbten Regionen
                           hover_name="Bundesland",      # Was im Tooltip als Haupttitel angezeigt wird
                           hover_data={'% BEV': ':.1f','schluessel': False}, # Optional: Formatiert die Anzeige der PKW_Bestand im Tooltip (als ganze Zahl)
                           #title="PKW-Bestand pro Bundesland in Deutschland" # Titel der Karte
                          )


fig.show()

In [ ]:
df_cars=df[["Bundesland","schluessel","sum_cars","e_cars","% BEV"]]

In [ ]:
df_cars_sorted=df_cars.sort_values(by="% BEV",ascending=False)

In [ ]:
df_cars_sorted.head(5)

In [ ]:
df_cars_sorted.columns=["Bundesland","schluessel","PKW_Gesamt","Anzahl_BEV","Anteil_BEV"]

In [ ]:
df_cars_sorted["Anteil_BEV"]=df_cars_sorted["Anteil_BEV"].round(2)

In [ ]:
df_cars_sorted

In [ ]:
df_bev=pd.read_csv("../../data/electric_vehicles/Bevoelkerung_Bundeslaender.csv",delimiter=";",encoding='latin-1')

In [ ]:
df_bev

In [ ]:
list1=df_cars_sorted["Bundesland"].to_list()
list2=df_bev["Bundesland"].to_list()

In [ ]:
check=[1 if land in list2 else 0 for land in list1]
all(check)       
    

In [ ]:
df_new=pd.merge(df_cars_sorted,df_bev,how="left",on="Bundesland")

In [ ]:
df_new

In [ ]:
df_new["BEV_pro_1000_Einwohner"]=(df_new["Anzahl_BEV"]/df_new["Einwohner"])*1000

In [ ]:
df_new["BEV_pro_1000_qkm"]=(df_new["Anzahl_BEV"]/df_new["Flaeche"])*1000

In [ ]:
df_new.to_csv("../../data/electric_vehicles/Anteil_BEV.csv",index=False)

In [ ]:
df_test=pd.read_csv("../../data/electric_vehicles/Anteil_BEV.csv")

In [ ]:
df_test

In [ ]:
df_test['schluessel'] = df_test['schluessel'].astype(str).str.zfill(2)

In [ ]:
df_test

In [ ]:
fig = px.choropleth_map(df_test,
                           geojson=geojson_bundeslaender_wgs84, # Die konvertierte GeoJSON-Datei
                           locations="schluessel", # Spalte im DataFrame, die den 2-stelligen Schlüssel enthält
                           featureidkey="properties.schluessel", # Pfad zum 'schluessel' im GeoJSON-Feature
                                                                 # (wie in Ihrer GeoJSON-Struktur gesehen: 'properties': {'schluessel': '06', ...})
                           color="BEV_pro_1000_Einwohner",               # Spalte, die die Farbe der Region bestimmt
                           color_continuous_scale="Blues",  # Farbskala (z.B. "Viridis", "Plasma", "Jet", "Greens", "Blues")
                           range_color=(df_test["BEV_pro_1000_Einwohner"].min(), df_test["BEV_pro_1000_Einwohner"].max()), # Stellen Sie sicher, dass die Farbskala die volle Bandbreite abdeckt
                          # mapbox_style="carto-positron",     # Basiskarte (z.B. "open-street-map", "carto-positron", "stamen-terrain")
                           zoom=5,                          # Zoom-Level für Deutschland (ca. 4.5 - 5.5)
                           center={"lat": 51.0, "lon": 10.0}, # Zentraler Punkt für Deutschland
                           opacity=0.8,                       # Deckkraft der eingefärbten Regionen
                           hover_name="Bundesland",      # Was im Tooltip als Haupttitel angezeigt wird
                           hover_data={'BEV_pro_1000_Einwohner': ':.0f','schluessel': False,"Anteil_BEV":':.1f',"PKW_Gesamt":":,.0f"}, # Optional: Formatiert die Anzeige der PKW_Bestand im Tooltip (als ganze Zahl)
                           #title="PKW-Bestand pro Bundesland in Deutschland" # Titel der Karte
                          )


fig.show()

In [ ]:
# Beispiel für Scatter Plot
fig_scatter = px.scatter(df_new, x='BEV_pro_1000_Einwohner', y='Anteil_BEV',
                         text='Bundesland',
                          title='Anteil BEV vs. BEV pro 1000 Einwohner')
fig_scatter.update_traces(textposition='top center') # Optional: Bundesland-Label über den Punkten
fig_scatter.show()

In [ ]:
# umwandel in long format zu besseren visualisierung
df_new.columns

In [ ]:
value_vars = ['PKW_Gesamt', 'Anzahl_BEV', 'Anteil_BEV',"BEV_pro_1000_Einwohner",'BEV_pro_1000_qkm']

# Die Spalte, die beibehalten werden soll (Identifikator)
id_vars = ['Bundesland']

df_long = pd.melt(df_new,
                  id_vars=id_vars,
                  value_vars=value_vars,
                  var_name='Kennzahl',  # Name für die neue Spalte, die die ursprünglichen Spaltennamen enthält
                  value_name='Wert')    # Name für die neue Spalte, die die Werte enthält

In [ ]:
df_long.head()

In [ ]:
df_filtered_for_hist = df_long[df_long["Kennzahl"].isin(["PKW_Gesamt", "Anzahl_BEV"])]


fig = px.histogram(
    df_filtered_for_hist,
    x="Wert",        # Die Werte der Kennzahlen auf der X-Achse
    facet_col="Kennzahl", # Eine separate Spalte/Diagramm für jede Kennzahl
    #facet_col_wrap=2, # Optional: Wenn du mehr als 2 Kennzahlen hast, um sie in mehreren Reihen anzuordnen
    labels={"Wert": "Anzahl / Bestand", "Kennzahl": "Metrik"},
    height=400,
    nbins=10,histnorm= # Anzahl der Bins im Histogramm
)
fig.show()

In [ ]:
df_test.columns

In [ ]:
land_max,anteil_max=df_test.loc[df_test['Anteil_BEV'].idxmax()][["Bundesland","Anteil_BEV"]]
land_min,anteil_min=df_test.loc[df_test['Anteil_BEV'].idxmin()][["Bundesland","Anteil_BEV"]]

In [ ]:
land_max_abs,anzahl_max=df_test.loc[df_test['Anzahl_BEV'].idxmax()][["Bundesland","Anzahl_BEV"]]


In [ ]:
land_min_abs,anzahl_min=df_test.loc[df_test['Anzahl_BEV'].idxmin()][["Bundesland","Anzahl_BEV"]]

In [ ]:
anzahl_max

In [ ]:
land_max_abs

In [ ]:
land_min_abs

In [ ]:
anzahl_min

In [ ]:
df_neuzu_bev=pd.read_csv("../../src/data/neuzulassung_bev.csv",delimiter=";",encoding='latin-1')

In [ ]:
df_neuzu_bev["anteil_bev"]=df_neuzu_bev["BEV"]/df_neuzu_bev["Gesamt"]*100

In [ ]:
df_neuzu_bev.head()

In [ ]:
fig=px.area(df_neuzu_bev,x="jahr",y="BEV",color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_xaxes(title_text="")
fig.update_yaxes(title_text="Neuzulassungen BEV")
fig.update_layout(hovermode="x unified")
fig.update_traces(hovertemplate= "<b>%{fullData.name}</b><br>" + # Name der Kennzahl (z.B. Benzin, Diesel)
                  "%{y:,.0f}<extra></extra>" # Wert mit Tausender-Trennzeichen und ohne Dezimalstellen
)

fig.show()

**BEV nach PLZ**


In [ ]:
with open("../../src/data/georef-germany-postleitzahl.geojson", 'r', encoding='utf-8') as f:
    geojson_plz= json.load(f)
    print("Geojson geladen")

In [ ]:
geojson_plz["features"][0]["properties"]

In [ ]:
data={"PLZ":[],"lon":[],"lat":[],"plz_name_long":[],"krs_name":[],"krs_code":[]}

for kreis in geojson_plz["features"]:
    data["PLZ"].append(kreis["properties"]["plz_code"])
    data["plz_name_long"].append(kreis["properties"]["plz_name_long"])
    data["krs_name"].append(kreis["properties"]["krs_name"])
    data["krs_code"].append(kreis["properties"]["krs_code"])
    data["lon"].append(kreis["properties"]["geo_point_2d"]["lon"])
    data["lat"].append(kreis["properties"]["geo_point_2d"]["lat"])
    

In [ ]:
df_geo=pd.DataFrame(data)
df_geo.head()
#df_geo.info()

In [ ]:
df_geo[df_geo["PLZ"]=="04574"]

In [ ]:
#df_geo[df_geo["krs_code"]=="10041"]


In [ ]:
len(df_geo["krs_code"].unique())

In [ ]:
df_geo["PLZ"].size

In [ ]:
df_geo["PLZ"].unique().size

In [ ]:
df_bev_plz=pd.read_csv("../../src/data/BEV_nach_plz.csv",delimiter=";",encoding='latin-1')
df_bev_plz["PLZ"] = df_bev_plz["PLZ"].astype(str).str.zfill(5)
df_bev_plz["Anteil_BEV"]=df_bev_plz["Bestand_BEV"]/df_bev_plz["Bestand_PKW"]*100

In [ ]:
df_bev_plz.head()

In [ ]:
df_bev_plz["PLZ"].size

In [ ]:
df_bev_plz["PLZ"].unique().size

In [ ]:
# merge

In [ ]:
bev_kreise=pd.merge(df_bev_plz,df_geo,how="left",on="PLZ")

In [ ]:
bev_kreise["Anteil_BEV"].idxmax()

In [ ]:
bev_kreise.loc[6556]

In [ ]:
bev_kreise.head()

In [ ]:
bev_kreise.isnull().sum()

In [ ]:
rows_with_any_null = bev_kreise[bev_kreise.isnull().any(axis=1)]
rows_with_any_null

In [ ]:
# PLZ 81248   (23 BEV bei 1494 PKW gelöscht)
# PLZ 99331   (59 BEV bei 2131 PKW gelöscht )

In [ ]:
bev_kreise["Anteil_BEV"].describe()

In [ ]:
bev_kreise.dropna(inplace=True)

In [ ]:
bev_kreise.isnull().sum()

In [ ]:
bev_kreise["krs_code"].unique().size

In [ ]:

fig = px.choropleth_map(bev_kreise,
                           geojson=geojson_plz, # Die konvertierte GeoJSON-Datei
                           locations="PLZ", # Spalte im DataFrame, die den 2-stelligen Schlüssel enthält
                           featureidkey="properties.plz_code", # Pfad zum 'schluessel' im GeoJSON-Feature
                                                                 # (wie in Ihrer GeoJSON-Struktur gesehen: 'properties': {'schluessel': '06', ...})
                           color="Anteil_BEV",               # Spalte, die die Farbe der Region bestimmt
                           color_continuous_scale="Blues",  # Farbskala (z.B. "Viridis", "Plasma", "Jet", "Greens", "Blues")
                           range_color=[0,5], # Stellen Sie sicher, dass die Farbskala die volle Bandbreite abdeckt
                           map_style="carto-positron",     # Basiskarte (z.B. "open-street-map", "carto-positron", "stamen-terrain")
                           zoom=5,                          # Zoom-Level für Deutschland (ca. 4.5 - 5.5)
                           center={"lat": 51.0, "lon": 10.0}, # Zentraler Punkt für Deutschland
                           opacity=0.8,                       # Deckkraft der eingefärbten Regionen
                           hover_name="plz_name_long",      # Was im Tooltip als Haupttitel angezeigt wird
                           hover_data={'Anteil_BEV': ':.1f',"Bestand_PKW":":,.0f",'PLZ': False,"Bestand_BEV":":,.0f"}, # Optional: Formatiert die Anzeige der PKW_Bestand im Tooltip (als ganze Zahl)
                           #title="PKW-Bestand pro Bundesland in Deutschland" # Titel der Karte
                          )


fig.update_traces( marker_line_width=0,marker_line_color='rgba(0,0,0,0)')

#"Anteil_BEV":':.1f'
fig.show()


In [ ]:
fig_density = px.density_map(bev_kreise,
                                lat="lat",
                                lon="lon",
                                z="Anteil_BEV", # Optional: Spalte zur Gewichtung der Dichte (z.B. höhere Werte ergeben intensivere Hitze)
                                radius=10, # Radius der Dichte-Schätzung in Pixeln (Standard ist 10)
                                zoom=5,
                                center={"lat": 51.0, "lon": 10.0},
                                map_style="carto-positron",
                                hover_name="PLZ",
                                hover_data={'Anteil_BEV': ':.1f'})

fig_density.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig_density.show()